# Check gradio working

In [5]:
import random
import gradio as gr

def alternatingly_agree(message, history):
    if len(history) % 2 == 0:
        return f"Yes, I do think that '{message}'"
    else:
        return "I don't think so"

iface = gr.ChatInterface(alternatingly_agree)
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [6]:
iface.close()

Closing server running on port: 7863


# Combining all to gradio

In [1]:
import gradio as gr
import ollama

d:\Raghu Studies\FinancialAdvisor\.src_financial_advisor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [14]:
from nltk.tokenize import word_tokenize
question_words = ["what", "why", "when", "where", 
             "name", "is", "how", "do", "does", 
             "which", "are", "could", "would", 
             "should", "has", "have", "whom", "whose", "don't"]

def check_question(question):
    question = question.lower()
    question = word_tokenize(question)
    if any(x in question for x in question_words):
        return 1
    else:
        return 0

'I am 30 years old Should I invest in CDSL?'

In [21]:
chat_list = []
merged_prompt = "ABOUT_ME:{}QUESTION:{}CONTEXT:{}"
def respond_chat(message, history):
    
    chat_list.append(message)
    is_question = check_question(message)
    if is_question:
        
        response = ollama.chat(model='mosrihari/unsloth_finance_alpaca', messages=[
                {"role": "user", "content": merged_prompt.format(
                        "I am 30 yeard old and I am planning on a long term investment",
                        "Should I invest in CDSL?",
                        "CDSL is a Central Depository where all the stocks when people invest go through that channel. Its a monopoly in India"
                    )},
                ])
        return response['message']['content']
    else:
        return "Ask question about where you want to invest"
iface = gr.ChatInterface(respond_chat)
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [22]:
iface.close()

Closing server running on port: 7862


# Consume from kafka OLLAMA

In [4]:
import ollama
from kafka import KafkaConsumer
import ast

In [5]:
consumer = KafkaConsumer(
    'summary',               # Topic name
    bootstrap_servers='localhost:9092',  # Kafka broker
    auto_offset_reset='earliest',        # Start at the earliest available message
    enable_auto_commit=True,             # Automatically commit offsets
    group_id='summary-group',      # Consumer group ID
    value_deserializer=lambda x: x.decode('utf-8')  # Decode message from bytes to string
    )
try:
    for message in consumer:
        # Print consumed message
        print(f"Message consumed: {message.value} of type {type(message)} from partition {message.partition}, offset {message.offset}")
        data = message.value
        data = ast.literal_eval(data)
        break
except:
    print("Stopping consumer...")
consumer.close()

Message consumed: {"about_me": "I am 30 years and I am looking for long term investment", "question": "Should I invest in facebook?", "context": [["Microsoft Corp (NASDAQ:MSFT) LinkedIn is deploying artificial intelligence to offer career advice and has introduced games to the platform, aiming to attract users more frequently and stimulate growth in the competitive social media landscape. LinkedIn has recently rolled out daily puzzles and AI features to craft articles that provide worker guidance, the Financial Times reports. The platform, traditionally a professional development tool, sees less frequent logins than entertainment-focused networks like Meta Platforms Inc&#8217;s (NASDAQ:META) Facebook and TikTok. Also Read: LinkedIn Co-Founder Criticizes Trump Media&#8217;s $6B Valuation Compared to Meta &#8211; What&#8217;s Going On? Editor-in-chief Daniel Roth told the Financial Times that the goal is to build a daily habit among users. LinkedIn reported a record engagement of 1.5 mil

In [6]:
data

{'about_me': 'I am 30 years and I am looking for long term investment',
 'question': 'Should I invest in facebook?',
 'context': [['Microsoft Corp (NASDAQ:MSFT) LinkedIn is deploying artificial intelligence to offer career advice and has introduced games to the platform, aiming to attract users more frequently and stimulate growth in the competitive social media landscape. LinkedIn has recently rolled out daily puzzles and AI features to craft articles that provide worker guidance, the Financial Times reports. The platform, traditionally a professional development tool, sees less frequent logins than entertainment-focused networks like Meta Platforms Inc&#8217;s (NASDAQ:META) Facebook and TikTok. Also Read: LinkedIn Co-Founder Criticizes Trump Media&#8217;s $6B Valuation Compared to Meta &#8211; What&#8217;s Going On? Editor-in-chief Daniel Roth told the Financial Times that the goal is to build a daily habit among users. LinkedIn reported a record engagement of 1.5 million content int

In [7]:
merged_prompt = "ABOUT_ME:{}QUESTION:{}CONTEXT:{}"

        
response = ollama.chat(model='mosrihari/unsloth_finance_alpaca', messages=[
        {"role": "user", "content": merged_prompt.format(
                data['about_me'],
                data['question'],
                data['summary']
            )},
        ])

In [8]:
response['message']['content']

" Long term investment in Facebook is viable due to its strong brand recognition and diversified revenue streams. The company's focus on innovation and user experience drives growth, making it an attractive long-term investment opportunity."